### P0 Code Generator for CLR
#### Simarpreet Singh, March 2017

The generated code is kept in memory and all code generation procedures continuously append to that code: procedure `genProgStart` initializes the generator, then `gen`-prefixed procedures are to be called for P0 constructs as in order in which they are recognized by a recursive descent parser, and finally procedure `genProgExit` returns the generated code in assembly language as a string in a format that can be read in by the SPIM simulator. The generation procedures are:
- `genBool`, `genInt`, `genRec`, `genArray`
- `genProgStart`, `genGlobalVars`, `genProgEntry`, `genProgExit`
- `genProcStart`, `genFormalParams`, `genLocalVars`, `genProcEntry`, `genProcExit`
- `genSelect`, `genIndex`, `genVar`, `genConst`, `genUnaryOp`, `genBinaryOp`, `genRelation`
- `genAssign`, `genActualPara`, `genCall`, `genRead`, `genWrite`, `genWriteln`
- `genSeq`, `genCond`, `genIfThen`, `genThen`, `genIfElse`, `genTarget`, `genWhile`

Errors in the code generator are reported by calling `mark` of the scanner. The data types of the symbol table are used to specify the P0 constructs for which code is to be generated.

In [1]:
import nbimporter
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, LE, \
     GE, NOT, mark
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool, Array, Record

importing Jupyter notebook from SC.ipynb


importing Jupyter notebook from ST.ipynb


Following variables determine the state of the code generator:

- `curlev` is the current level of nesting of P0 procedures
- `regs` is the set of available MIPS registers for expression evaluation
- `label` is a counter for generating new labels
- `asm` is a list of triples; each triple consists of three (possibly empty) strings:
  - a label
  - an instruction, possibly with operands
  - a target (for branch and jump instructions)

Procedure `genProgStart()` initializes these variables. Registers `$t0` to `$t9` are used as general-purpose registers.

In [2]:
def genProgStart():
    global asm, curlev, label, regs
    asm, curlev, label = [], 0, 0
    regs = {'$t0', '$t1', '$t2', '$t3', '$t4', '$t5', '$t6', '$t7', '$t8'}
    
    global instrs, tablvl, glvars, classInstrs, classes, externalLibInstrs, proclvl, procs, mainClassDefaultLines
    instrs, tablvl, glvars, proclvl, procs, mainClassDefaultLines = [], 0, [], 0, [], []
    classInstrs = []
    classes = {}
    externalLibInstrs = ['.assembly extern mscorlib{}', '.assembly \'main\'{}']
    putInstr2('.class private auto ansi beforefieldinit MainClass')
    putInstr2('extends [mscorlib]System.Object', 1)
    putInstr2('{')
    tablvl+=1

Reserved registers are `$0` for the constant `0`, `$fp` for the frame pointer, `$sp` for the stack pointer, and `$ra` for the return address (dynamic link).

In [3]:
R0 = '$0'; FP = '$fp'; SP = '$sp'; LNK = '$ra'

def obtainReg():
    if len(regs) == 0: mark('out of registers'); return R0
    else: return regs.pop()

def releaseReg(r):
    if r not in (R0, SP, FP, LNK): regs.add(r)

In [4]:
def putLab(lab, instr = ''):
    """Emit label lab with optional instruction; lab may be a single
    label or a list of labels"""
    if type(lab) == list:
        for l in lab[:-1]: asm.append((l, '', ''))
        asm.append((lab[-1], instr, ''))
    else: asm.append((lab, instr, ''))

def putInstr(instr, target = '', ):
    """Emit an instruction"""
    asm.append(('', instr, target))

def putInstr2(instr, tabLevel = 0):
    if proclvl > 0:
        current_top_proc = getCurrentProc()
        current_top_proc[4].append(instr)
    else:
        instrs.append((instr, proclvl+tablvl+tabLevel))

def putOp(op, a, b, c):
    """Emit instruction op with three operands, a, b, c; c can be register or immediate"""
    putInstr(op + ' ' + a + ', ' + b + ', ' + str(c))

def putBranchOp(op, a, b, c):
    putInstr(op + ' ' + a + ', ' + b, str(c))

def putMemOp(op, a, b, c):
    """Emit load/store instruction at location or register b + offset c"""
    if b == R0: putInstr(op + ' ' + a + ', ' + str(c))
    else: putInstr(op + ' ' + a + ', ' + str(c) + '(' + b + ')')

Following procedures "generate code" for all P0 types by determining the size of objects and store in the `size` field.
- Integers and booleans occupy 4 bytes
- The size of a record is the sum of the sizes of its field; the offset of a field is the sum of the size of the preceding fields
- The size of an array is its length times the size of the base type.

In [5]:
def genBool(b):
    b.size = 4; return b

def genInt(i):
    i.size = 4; return i

def genRec(r):
    # r is Record
    s = 0
    for f in r.fields:
        f.offset, s = s, s + f.tp.size
    r.size = s
    return r

def genArray(a):
    # a is Array
#     createTypeClass('hello', 'something')
    a.size = a.length * a.base.size
    return a

For each global variable, `genGlobalVars(sc, start)` generates the assembler _directive_ `.space`, consisting of the identifier as the label and the size of the variable as the operand. The parameter `sc` contains the top scope with all declarations parsed so far; only variable declarations from index `start` on in the top scope are considered. As MIPS instructions are not allowed to be identifiers, all variables get `_` as suffix to avoid a name clash.

In [6]:
def genGlobalVars(sc, start):
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            sc[i].reg, sc[i].adr = R0, sc[i].name + '_'
            putLab(sc[i].adr, '.space ' + str(sc[i].tp.size))
            
            type_name = getClRTypeString(sc[i].tp)
            
            glvars.append((sc[i].tp, sc[i].name, R0))
            
            putInstr2('.field static '+type_name+' '+sc[i].name)
            
            if 'class ' in type_name:
                mainClassDefaultLines.append('newobj instance void '+type_name+'::\'.ctor\'()')
                mainClassDefaultLines.append('stsfld '+type_name+' MainClass::'+sc[i].name)

Procedure `genProgEntry(ident)` takes the program's name as a parameter. Directives for marking the beginning of the main program are generated; the program's name is it not used. 

In [7]:
def genProgEntry(ident):
    global tablvl
    putInstr2('.method private static hidebysig default void Main (string[] args)')
    putInstr2('cil managed', 1)
    putInstr2('{')
    tablvl+=1
    putInstr2('.entrypoint')
    
    
    putInstr('.text')
    putInstr('.globl main')
    putInstr('.ent main')
    putLab('main')

Procedure `genProgExit(x)` takes parameter `x` with the result of previous `gen-` calls, generates code for exiting the program, directives for marking the end of the main program, and returns the complete assembly code.

In [8]:
def assembly(l, i, t):
    """Convert label l, instruction i, target t to assembly format"""
    return (l + ':\t' if l else '\t') + i + (', ' + t if t else '')

def addTabs(i, t):
    return ("\t"*t)+i

def genProgExit(x):
    global tablvl
    putInstr2('ret')
    putInstr2('}', -1)
    if len(mainClassDefaultLines) > 0:
        tablvl-=1
        putMainClassDefaultMethod(mainClassDefaultLines)
        tablvl+=1
    putInstr2('}', -2)
    putClassInstrs()
    putExternalLibInstrs()
    return '\n'.join(addTabs(i, t) for (i, t) in instrs)
#     return '\n'.join(assembly(l, i, t) for (l, i, t) in asm)

Procedure `newLabel()` generates a new unique label on each call.

In [9]:
def newLabel():
    global label
    label += 1
    return 'L' + str(label)

The code generator _delays the generation of code_ until it is clear that no better code can be generated. For this, the not-yet-generated result of an expressions and the location of a variable is stored in _items_. In addition to the symbol table types `Var`, `Ref`, `Const`, the generator uses two more item types:
- `Reg(tp, reg)` for integers or boolean values stored in a register; the register can be `$0` for constants `0` and `false`
- `Cond(cond, left, right)` for short-circuited Boolean expressions with two branch targets. The relation `cond` must be one of `'EQ'`, `'NE'`, `'LT'`, `'GT'`, `'LE'`, `'GE'`. The operands `left`, `right` are either registers or constants, but one has to be a register. The result of the comparison is represented by two branch targets, stored as fields, where the evaluation continues if the result of the comparison is true or false. The branch targets are lists of unique labels, with targets in each list denoting the same location. If `right` is `$0`, then `'EQ'` and `'NE'` for `cond` can be used for branching depending on whether `left` is `true` or `false`.

In [10]:
class Reg:
    def __init__(self, tp, reg):
        # tp is Bool or Int
        self.tp, self.reg = tp, reg

class Cond:
    # labA, labB are lists of branch targets for when the result is true or false
    def __init__(self, cond, left, right):
        self.tp, self.cond, self.left, self.right = Bool, cond, left, right
        self.labA, self.labB = [newLabel()], [newLabel()]

Procedure `loadItemReg(x, r)` generates code for loading item `x` to register `r`, assuming `x` is `Var`, `Const`, or `Reg`. If a constant is too large to fit in 16 bits immediate addressing, an error message is generated.

In [11]:
def testRange(x):
    if x.val >= 0x8000 or x.val < -0x8000: mark('value too large')

def loadItemReg(x, r):
    global glvars
    if type(x) == Var: 
        if hasattr(x, 'reg'):
            putMemOp('lw', r, x.reg, x.adr); releaseReg(x.reg)
    elif type(x) == Const:
        loadConstCLR(x.val)
        
        testRange(x); putOp('addi', r, R0, x.val)
    elif type(x) == Reg: # move to register r
        putOp('add', r, x.reg, R0)
    else: assert False

Procedure `loadItem(x)` generates code for loading item `x`, which has to be `Var` or `Const`, into a new register and returns a `Reg` item; if `x` is `Const` and has value `0`, no code is generated and register `R0` is used instead. For procedure `loadBool(x)`, the type of item `x` has to be `Bool`; if `x` is not a constant, it is loaded into a register and a new `Cond` item is returned.

In [12]:
def loadItem(x):
    if type(x) == Const and x.val == 0:
        r = R0 # use R0 for "0"
        loadConstCLR(0)
    else: 
        r = obtainReg()
        loadItemReg(x, r)
    return Reg(x.tp, r)

def loadBool(x):
    if type(x) == Const and x.val == 0: r = R0 # use R0 for "false"
    else: r = obtainReg(); loadItemReg(x, r)
    return Cond(NE, r, R0)

Procedure `put(cd, x, y)` generates code for `x op y`, where `op` is an operation with mnemonic `cd`. Items `x`, `y` have to be `Var`, `Const`, `Reg`. An updated item `x` is returned.

In [13]:
def put(cd, x, y):
    if type(x) != Reg: 
        x = loadItem(x)
    if x.reg in (R0, '$a0', '$a1', '$a2', '$a3'): # find new destination register
        r = x.reg; x.reg = obtainReg()
    else: r = x.reg # r is source, x.reg is destination
    if type(y) == Const:
        testRange(y); putOp(cd, x.reg, r, y.val)
    else:
        if type(y) != Reg: y = loadItem(y)
        putOp(cd, x.reg, r, y.reg); releaseReg(y.reg)
    return x

Procedures `genVar`, `genConst`, `genUnaryOp`, `genBinaryOp`, `genRelation`, `genSelect`, and `genIndex` generate code for expressions (e.g. right hand side of assignments) and for  locations (e.g. left hand side of assignments).

Procedure `genVar(x)` allows `x` to refer to a global variable, local variable, or procedure parameter: the assumption is that `x.reg` (which can be `R0`) and `x.adr` (which can be 0) refer to the variable. References to variables on intermediate level is not supported. For global variables, the reference is kept symbolic, to be resolved later by the assembler. Item `x` is `Var` or `Ref`; if it is `Ref`, the reference is loaded into a new register. A new `Var` item with the location is returned.

In [14]:
"""  
def genVar(x): # version not supporting parameters in registers
    if 0 < x.lev < curlev: mark('level!')
    y = Var(x.tp); y.lev = x.lev
    if type(x) == Ref: # reference is loaded into register
        y.reg, y.adr = obtainReg(), 0 # variable at M[y.reg]
        putMemOp('lw', y.reg, x.reg, x.adr)
    elif type(x) == Var:
        y.reg, y.adr = x.reg, x.adr 
    else: assert False
    return y
"""
def genVar(x): # version supporting parameters in registers
    if 0 < x.lev < curlev: mark('level!')
    if type(x) == Ref:
        y = Var(x.tp); y.lev = x.lev
        if hasattr(x, 'reg'):
            if x.reg in ('$a0', '$a1', '$a2', '$a3'): # reference already in register, use it
                y.reg, y.adr = x.reg, 0 # variable at M[y.reg]
            else: # reference is loaded into register
                y.reg, y.adr = obtainReg(), 0 # variable at M[y.reg]
                putMemOp('lw', y.reg, x.reg, x.adr)
    elif type(x) == Var:
        if x.reg in ('$a0', '$a1', '$a2', '$a3'): # value already in register, use it
            y = Reg(x.tp, x.reg) #; y.lev, x.adr = x.lev, x.adr
        else:
            y = Var(x.tp); y.lev, y.reg, y.adr, y.name = x.lev, x.reg, x.adr, x.name
    else: assert False
    return y

Procedure `genConst(x)` does not need to generate any code.

In [15]:
def genConst(x):
    # x is Const
    return x

Procedure `genUnaryOp(op, x)` generates code for `op x` if `op` is `MINUS`, `NOT` and `x` is `Int`, `Bool`; if `op` is `AND`, `OR`, item `x` is the first operand. If it is not already a `Cond` item, it is made so which is loaded into a register. A branch instruction is generated for `OR` and a branch instruction with a negated condition for `AND`.

In [16]:
def negate(cd):
    return {EQ: NE, NE: EQ, LT: GE, LE: GT, GT: LE, GE: LT}[cd]

def condOp(cd):
    return {EQ: 'beq', NE: 'bne', LT: 'blt', LE: 'ble', GT: 'bgt', GE: 'bge'}[cd]

def genUnaryOp(op, x):
    if op == MINUS: # subtract from 0
        if type(x) == Var: x = loadItem(x)
        putOp('sub', x.reg, R0, x.reg)
    elif op == NOT: # switch condition and branch targets, no code
        if type(x) != Cond: x = loadBool(x)
        x.cond = negate(x.cond); x.labA, x.labB = x.labB, x.labA
    elif op == AND: # load first operand into register and branch
        if type(x) != Cond: x = loadBool(x)
        putBranchOp(condOp(negate(x.cond)), x.left, x.right, x.labA[0])
        releaseReg(x.left); releaseReg(x.right); putLab(x.labB)
    elif op == OR: # load first operand into register and branch
        if type(x) != Cond: x = loadBool(x)
        putBranchOp(condOp(x.cond), x.left, x.right, x.labB[0])
        releaseReg(x.left); releaseReg(x.right); putLab(x.labA)
    else: assert False
    return x

Procedure `genBinaryOp(op, x, y)` generates code for `x op y` if `op` is `PLUS`, `MINUS`, `TIMES`, `DIV`, `MOD`. If `op` is `AND`, `OR`, operand `y` is made a `Cond` item it if is not so already and the branch targets are merged. 

In [17]:
def genBinaryOp(op, x, y):
    
    
    if type(x) == Const:
        loadConstCLR(x.val)
    else:
        if x.tp in {Int, Bool}:
            loadVariableCLR(x)

    if type(y) == Const:
        loadConstCLR(y.val)
    else:
        if y.tp in {Int, Bool}:
            loadVariableCLR(y)
        
        
    if type(x) == Const:
        temp = y
        x = y
        y = temp
        
    if op == PLUS: 
        putInstr2('add')
        y = put('add', x, y)
    elif op == MINUS: 
        putInstr2('sub')
        y = put('sub', x, y)
    elif op == TIMES:
        putInstr2('mul')
        y = put('mul', x, y)
    elif op == DIV:
        putInstr2('div')
        y = put('div', x, y)
    elif op == MOD:
        putInstr2('rem')
        y = put('mod', x, y)
    elif op == AND: # load second operand into register 
        if type(y) != Cond: y = loadBool(y)
        y.labA += x.labA # update branch targets
    elif op == OR: # load second operand into register
        if type(y) != Cond: y = loadBool(y)
        y.labB += x.labB # update branch targets
    else: assert False
    return y

Procedure `genRelation(op, x, y)` generates code for `x op y` if `op` is `EQ`, `NE`, `LT`, `LE`, `GT`, `GE`. Items `x` and `y` cannot be both constants. A new `Cond` item is returned.

In [18]:
def genRelation(op, x, y):
    if type(x) != Reg: x = loadItem(x)
    if type(y) != Reg: y = loadItem(y)
    return Cond(op, x.reg, y.reg)

Procedure `genSelect(x, f)` "generates code" for `x.f`, provided `f` is in `x.fields`. Only `x.adr` is updated, no code is generated. An updated item is returned.

In [19]:
def genSelect(x, f):
    
    from_class = 'MainClass'
#     if f.tp in classes.values():
#         from_class = getClRTypeString(f.tp)[6:]

    if hasattr(x, 'name'):
        putInstr2('ldsfld '+getClRTypeString(x.tp)+' '+from_class+'::'+x.name)
    
    x.name = f.name
    
    x.selectorClass = x.tp
    x.tp = f.tp
    if hasattr(x, 'adr'):
        x.adr = x.adr + f.offset if type(x.adr) == int else \
                            x.adr + '+' + str(f.offset)
    
    
#     x.tp = x.name
#     print(x)
    return x

Procedure `genIndex(x, y)` generates code for `x[y]`, assuming `x` is `Var` or `Ref`, `x.tp` is `Array`, and `y.tp` is `Int`. If `y` is `Const`, only `x.adr` is updated and no code is generated, otherwise code for array index calculation is generated.

In [20]:
def genIndex(x, y):  
    if type(y) == Const:
        loadConstCLR(y.val - x.tp.lower)
        offset = (y.val - x.tp.lower) * x.tp.base.size
        if hasattr(x, 'adr'):
            x.adr = x.adr + (offset if type(x.adr) == int else '+' + str(offset))
    else:
        if type(y) != Reg:
            x.varName = y.name
            y = loadItem(y)
        if hasattr(x, 'reg') and hasattr(y, 'reg'):    
            putOp('sub', y.reg, y.reg, x.tp.lower)
            putOp('mul', y.reg, y.reg, x.tp.base.size)
            if x.reg != R0:
                putOp('add', y.reg, x.reg, y.reg); releaseReg(x.reg)
            x.reg = y.reg
    x.tp = x.tp.base
    x.isFromArray = True
    return x

Procedure `genAssign(x, y)` generates code for `x := y`, provided `x` is `Var`. Item `x` is loaded into a register if it is not already there; if `x` is `Cond`, then either `0` or `1` is loaded into a register.

In [21]:
def genAssign(x, y):
    if type(x) == Var:
        
        
        if type(y) == Cond:
            putBranchOp(condOp(negate(y.cond)), y.left, y.right, y.labA[0])
            releaseReg(y.left); releaseReg(y.right); r = obtainReg()
            putLab(y.labB); putOp('addi', r, R0, 1) # load true
            lab = newLabel()
            putInstr('b', lab)
            putLab(y.labA); putOp('addi', r, R0, 0) # load false 
            putLab(lab)
        elif type(y) != Reg: 
            y = loadItem(y); r = y.reg
            
        else: 
            r = y.reg
            
        storeVariableCLR(x)
            
        if hasattr(x, 'reg'):
            putMemOp('sw', r, x.reg, x.adr) 
        
        releaseReg(r)
        
        
        
    else: assert False

The procedure calling convention is as follows:
- last parameter at `0($fp)`, 2nd last at `4($fp)`, ...
- previous frame pointer at `-4($fp)`
- return address at `-8($fp)`
- 1st local at `-12($fp)`, ...

The Stack pointer `$sp` points to last used location on the stack.

On procedure entry:
- caller pushes 1st parameter at `-4($sp)`, 2nd at `-8($sp)`, ...
- caller calls callee
- callee saves `$fp` at `$sp - parameter size - 4`
- callee saves `$ra` at `$sp - parameter size - 8`
- callee sets `$fp` to `$sp - parameter size`
- callee sets `$sp` to `$fp - local var size - 8`

On procedure exit:
- callee sets `$sp` to `$fp + parameter size`
- callee loads `$ra` from `$fp - 8`
- callee loads `$fp` from `$fp - 4`
- callee returns

For each local variable, `genLocalVars(sc, start)` updates the entry of the variable with the FP-relative address and returns their total size. The parameter `sc` contains the top scope with all local declarations parsed so far; only variable declarations from index `start` on in the top scope are considered.

In [22]:
def genLocalVars(sc, start):
    s = 0 # local block size
    if proclvl > 0:
        for i in range(len(sc)):
            if type(sc[i]) == Var:
                s = s + sc[i].tp.size
                sc[i].reg, sc[i].adr = FP, - s - 8
                current_top_proc = getCurrentProc()
                current_top_proc[3].append(sc[i])
    else:
        putInstr2('.locals init (')
        for i in range(start, len(sc)):
            if type(sc[i]) == Var:
                s = s + sc[i].tp.size
                sc[i].reg, sc[i].adr = FP, - s - 8

                type_name = getClRTypeString(sc[i].tp)

                if i == len(sc)-1:
                    putInstr2(type_name+' V_'+str(i)+')', 1)
                else:
                    putInstr2(type_name+' V_'+str(i)+',', 1)
            
    return s

Procedure `genProcStart()` generate the directive for starting instructions.

In [23]:
def genProcStart():
    global curlev
    curlev = curlev + 1
    putInstr('.text')

Procedure `genFormalParams(sc)` determines the FP-relative address of all parameters in the list `sc` of procedure parameters. Each parameter must be of type `Int` or `Bool` or must be a reference parameter.

In [24]:
def genFormalParams(ident, sc):
    global proclvl
    arguments = []
    
    n = len(sc) # parameter block length
    for i in range(n):
        if sc[i].tp in (Int, Bool) or type(sc[i].tp) == Ref:
            sc[i].reg, sc[i].adr = FP, (n - i - 1) * 4
            
        if sc[i].tp is Int:
            arguments.append(sc[i])
        elif sc[i].tp is Bool:
            arguments.append(sc[i])
        elif sc[i].tp in classes.values():
            arguments.append(sc[i])
            
        else: mark('no structured value parameters')

    
    if proclvl > 0:
        current_top_proc = getCurrentProc()
        current_top_proc[1].append((ident, [], arguments, [], []))
    else:
        procs.append((ident, [], arguments, [], []))
    
    proclvl = proclvl+1
    
    return n * 4

Procedures `genProcEntry(ident, parsize, localsize)` and `genProcExit(x, parsize, localsize)` generate the procedure prologue and epilogue.

In [25]:
def genProcEntry(ident, parsize, localsize):
    putInstr('.globl ' + ident)            # global declaration directive
    putInstr('.ent ' + ident)              # entry point directive
    putLab(ident)                          # procedure entry label
    putMemOp('sw', FP, SP, - parsize - 4)  # push frame pointer
    putMemOp('sw', LNK, SP, - parsize - 8) # push return address
    putOp('sub', FP, SP, parsize)          # set frame pointer
    putOp('sub', SP, FP, localsize + 8)    # set stack pointer

def genProcExit(x, parsize, localsize):
    global curlev
    curlev = curlev - 1
    putOp('add', SP, FP, parsize) # restore stack pointer
    putMemOp('lw', LNK, FP, - 8)  # pop return address
    putMemOp('lw', FP, FP, - 4)   # pop frame pointer
    putInstr('jr $ra')            # return
    
    global proclvl
    proclvl = proclvl - 1
#     putInstr2('}')
    if proclvl == 0:
        putProcedure(procs[len(procs)-1])

Procedure `genActualPara(ap, fp, n)` assume that `ap` is an item with the actual parameter, `fp` is the entry for the formal parameter, and `n` is the parameter number. The parameters are pushed SP-relative on the stack. The formal parameter is either `Var` or `Ref`.

In [26]:
def genActualPara(ap, fp, n):
    if type(fp) == Ref:  #  reference parameter, assume p is Var
        if ap.adr != 0:  #  load address in register
            r = obtainReg(); putMemOp('la', r, ap.reg, ap.adr)
        else: r = ap.reg  #  address already in register
        putMemOp('sw', r, SP, - 4 * (n + 1)); releaseReg(r)
    else:  #  value parameter
        if type(ap) != Cond:
            if type(ap) != Reg: ap = loadItem(ap)
            putMemOp('sw', ap.reg, SP, - 4 * (n + 1)); releaseReg(ap.reg)
        else: mark('unsupported parameter type')
    return (ap, fp, n)

Procedure `genCall(pr, ap)` assumes `pr` is `Proc` and `ap` is a list of actual parameters.

In [27]:
def genCall(pr, ap):
    putInstr('jal', pr.name)
    
    proc_ret = findProcByLevel(procs, pr.name, pr.lev)
    if proc_ret:
        trace_string = ''
        if len(proc_ret[1]) > 0:
            trace_string += '/'
            trace_string +='/'.join(proc_ret[1])
        if(len(ap) > 0):
            for a in ap:
                loadVariableCLR(a[0])
        putInstr2('call void class MainClass'+trace_string+'/'+pr.name+'::main_proc('+', '.join(getClRTypeString(l.tp) for (l, m, n) in ap)+')')

Procedures `genRead(x)`, `genWrite(x)`, `genWriteln()` generate code for SPIM-defined "syscalls"; `genRead(x)` and assumes that `x` is `Var` and `genWrite(x)` assumes that `x` is `Ref`, `Var`, `Reg`.  

In [28]:
def genRead(x):
    putInstr('li $v0, 5'); putInstr('syscall')
    putMemOp('sw', '$v0', x.reg, x.adr)
    
#     putInstr2('.locals init (int32 V_0)')
    putInstr2('call string class [mscorlib]System.Console::ReadLine()')
    putInstr2('call int32 class [mscorlib]System.Convert::ToInt32(string)')
    type_name = getClRTypeString(x.tp)
    putInstr2('stsfld '+type_name+' MainClass::'+x.name)
    
    

def genWrite(x):
    loadItemReg(x, '$a0'); putInstr('li $v0, 1'); putInstr('syscall')
        
    type_name = getClRTypeString(x.tp)
    
    if type(x) != Reg:
        loadVariableCLR(x)
    putInstr2('call void class [mscorlib]System.Console::Write('+type_name+')')
        

def genWriteln():
    putInstr('li $v0, 11'); putInstr("li $a0, '\\n'"); putInstr('syscall')
    putInstr2('call void class [mscorlib]System.Console::WriteLine()')

For control structures:
- `genSeq(x, y)` generates `x ; y`, assuming `x`, `y` are statements
- `genCond(x)` generates code for branching on `x`, assuming that `x` is of type `Bool`
- `genIfThen(x, y)` generates code for `y` in `if x then y`
- `genThen(x, y)` generates code for `y` in `if x then y else z`
- `genIfElse(x, y, z)` generates code for `z` in `if x then y else z`
- `genTarget()` generates and returns a target for backward branches
- `genWhile(lab, x, y)` generates code for `y` in `while x do y`, assuming that target `lab` was generated before `x`.

In [29]:
def genSeq(x, y):
    pass

def genCond(x):
    if type(x) != Cond: x = loadBool(x)
    putBranchOp(condOp(negate(x.cond)), x.left, x.right, x.labA[0])
    releaseReg(x.left); releaseReg(x.right); putLab(x.labB)
    return x

def genIfThen(x, y):
    putLab(x.labA)

def genThen(x, y):
    lab = newLabel()
    putInstr('b', lab)
    putLab(x.labA); 
    return lab

def genIfElse(x, y, z):
    putLab(y)

def genTarget():
    lab = newLabel()
    putLab(lab)
    return lab

def genWhile(lab, x, y):
    putInstr('b', lab)
    putLab(x.labA); 

In [30]:
def loadConstCLR(c):
    if(c <= 8):
        putInstr2('ldc.i4.'+str(c))
    else:
        putInstr2('ldc.i4.s '+str(hex(c)))

In [31]:
def createTypeClass(ident, x):
    classes[ident] = x.val
    classInstr = []
    defaultMethodLines = []
    classInstr.append(('.class private auto ansi beforefieldinit '+ident, 0))
    classInstr.append(('extends [mscorlib]System.Object', 1))
    classInstr.append(('{', 0))
    if type(x.val) is Array:
        defaultMethodLines += putArrayField(ident, x.val, classInstr)
    elif type(x.val) is Record:
        for field in x.val.fields:
            field.of = ident
            type_name = getClRTypeString(field.tp)
            field_name = field.name
            if type_name:
                classInstr.append(('.field  public '+type_name+' '+field_name, 1))
                if 'class ' in type_name:
                    defaultMethodLines.append('ldarg.0')
                    defaultMethodLines.append('newobj instance void '+type_name+'::\'.ctor\'()')
                    defaultMethodLines.append('stfld '+type_name+' '+ident+'::'+field_name)
            elif type(field.tp) is Array:
                defaultMethodLines += putArrayField(ident, field.tp, classInstr, field_name)
    putTypeClassDefaultMethod(classInstr, defaultMethodLines)
    classInstr.append(('}', 0))
    classInstrs.append(classInstr)
    

In [32]:
def putArrayField(ident, x, classInstr, array_name = 'arr'):
    defaultMethodLines = []
    
    sys_type_name, type_class_name = None, None
    
    type_name = getClRTypeString(x.base)
    if type_name is 'int32':
        sys_type_name = 'Int32'
    elif type_name is 'bool':
        sys_type_name = 'Boolean'
    elif 'class ' in type_name:
        type_class_name = type_name[6:]
        
    classInstr.append(('.field  public '+type_name+'[] '+array_name, 1))

    defaultMethodLines.append('ldarg.0')
    defaultMethodLines.append('ldc.i4.s '+str(hex(x.length)))
    if sys_type_name:
        defaultMethodLines.append('newarr [mscorlib]System.'+sys_type_name)
    if type_class_name:
        defaultMethodLines.append('newarr '+type_class_name)
    defaultMethodLines.append('stfld '+type_name+'[] '+ident+'::'+array_name)
    return defaultMethodLines
    
        

In [33]:
def putTypeClassDefaultMethod(classInstr, defaultMethodLines):
    classInstr.append(('.method public hidebysig specialname rtspecialname', 1))
    classInstr.append(('instance default void \'.ctor\' ()  cil managed ', 2))
    classInstr.append(('{', 1))
    classInstr.append(('.maxstack 8', 2))
    for defaultMethodLine in defaultMethodLines:
        classInstr.append((defaultMethodLine, 2))
    classInstr.append(('ldarg.0', 2))
    classInstr.append(('call instance void object::\'.ctor\'()', 2))
    classInstr.append(('ret', 2))
    classInstr.append(('}', 1))
    

In [34]:
def putMainClassDefaultMethod(defaultMethodLines):
    putInstr2('.method private static hidebysig specialname rtspecialname')
    putInstr2('default void \'.cctor\' ()  cil managed ', 1)
    putInstr2('{')
    putInstr2('.maxstack 8', 1)
    for defaultMethodLine in defaultMethodLines:
        putInstr2(defaultMethodLine, 1)
    putInstr2('ret', 1)
    putInstr2('}')

In [35]:
def putClassInstrs():
    if len(classInstrs) > 0:
        instrs.insert(0,('\n', 0))
        for class_ in reversed(classInstrs):
            for (i, v) in reversed(class_):
                instrs.insert(0,(i, v))

In [36]:
def putExternalLibInstrs():
    instrs.insert(0,('\n', 0))
    for i in reversed(externalLibInstrs):
        instrs.insert(0,(i, 0))

In [37]:
def putProcedure(proc):
    global tablvl
    name = proc[0]
    nested_procs = proc[1]
    arguments = proc[2]
    proc_locals = proc[3]
    proc_instrs = proc[4]
    putInstr2('\n')
    putInstr2('.class nested private auto ansi beforefieldinit '+name)
    putInstr2('extends [mscorlib]System.Object', 1)
    putInstr2('{')
    putInstr2('.method public static hidebysig', 1)
    putInstr2('default void main_proc ('+', '.join(getClRTypeString(sc.tp)+' '+sc.name for sc in arguments)+')  cil managed', 2)
    putInstr2('{',1)
    if len(proc_locals) > 0:
        putInstr2('.locals init (', 2)
    for i in range(len(proc_locals)):
        if i == len(proc_locals)-1:
            putInstr2(getClRTypeString(proc_locals[i].tp)+' V_'+str(i)+')', 3)
        else:
            putInstr2(getClRTypeString(proc_locals[i].tp)+' V_'+str(i)+',', 3)
    for i in range(len(proc_instrs)):
        putInstr2(proc_instrs[i], 2)
    putInstr2('ret', 2)
    putInstr2('}',1)
    for i in range(len(nested_procs)):
        tablvl+=1
        putProcedure(nested_procs[i])
        tablvl-=1
    putInstr2('}')
    putInstr2('\n')

In [38]:
def getCurrentProc():
    if proclvl > 0:
        current_top_proc = procs[len(procs)-1]
        for i in range(proclvl-1):
            current_top_proc = current_top_proc[1][len(current_top_proc[1])-1]
        return current_top_proc
    return None

In [39]:
def getClRTypeString(tp):
    if tp is Int:
        return 'int32'
    elif tp is Bool:
        return 'bool'
    elif tp in classes.values():
        return 'class '+list(classes.keys())[list(classes.values()).index(tp)]
    else:
        return None

In [40]:
def findProcByLevel(procs_, name, lvl, trace = []):
    for i in range(len(procs_)):
        proc = procs_[i]
        if lvl == 0:
            if proc[0] == name:
                return (proc, trace)
        elif lvl > 0:
            findProcSublevel = findProcByLevel(proc[1], name, lvl-1, trace+[proc[0]])
            if findProcSublevel:
                return findProcSublevel
    return None

In [41]:
def loadVariableCLR(x):
    if type(x) == Const:
        print(x)
    elif hasattr(x, 'isFromArray') and x.lev == 0:
        putInstr2('ldelem.i4')
    elif x.lev > 0:
        cur_proc = getCurrentProc()
        
        lv_index = 0
        found_lv = False
        
        if hasattr(x, 'varName'):
            for lv in cur_proc[3]:
                if lv.name == x.varName and lv.tp == x.tp:
                    putInstr2('ldloc.'+str(lv_index))
                    found_lv = True
                    break
                lv_index+=1
        elif hasattr(x, 'name'):
            for lv in cur_proc[3]:
                if hasattr(x, 'selectorClass'):
                    if lv.name == x.name and lv.tp == x.selectorClass:
                        putInstr2('ldloc.'+str(lv_index))
                        found_lv = True
                        break
                elif lv.name == x.name and lv.tp == x.tp:
                    putInstr2('ldloc.'+str(lv_index))
                    found_lv = True
                    break
                lv_index+=1
                
        if not found_lv:
            argv_index = 0
            for argv in cur_proc[2]:
                if hasattr(x, 'selectorClass'):
                    if argv.tp == x.selectorClass:
                        putInstr2('ldarg.'+str(argv_index))
                        putInstr2('ldfld '+getClRTypeString(x.tp)+' '+getClRTypeString(x.selectorClass)[6:]+'::'+x.name)
                        break
#                 if argv.name == x.name and argv.tp == x.tp:
                elif argv.tp == x.tp:
                    putInstr2('ldarg.'+str(argv_index))
                    break
                argv_index+=1
        
        if hasattr(x, 'isFromArray'):
            putInstr2('ldelem.i4')
    elif hasattr(x, 'selectorClass'):
        if hasattr(x, 'name'):
            putInstr2('ldfld '+getClRTypeString(x.tp)+' '+getClRTypeString(x.selectorClass)[6:]+'::'+x.name)
    else:
        putInstr2('ldsfld '+getClRTypeString(x.tp)+' MainClass::'+x.name)


In [42]:
def storeVariableCLR(x):
    if hasattr(x, 'isFromArray'):
        putInstr2('stelem.i4')
    elif hasattr(x, 'selectorClass'):
        putInstr2('stfld '+getClRTypeString(x.tp)+' '+getClRTypeString(x.selectorClass)[6:]+'::'+x.name)
    elif proclvl > 0:
        current_top_proc = getCurrentProc()
        if hasattr(x, 'name'):
            localVarIndex = None
            li = 0
            for localprocvar in current_top_proc[3]:
                if localprocvar.tp is x.tp and localprocvar.name is x.name:
                    localVarIndex = li
                    break
                li+=1
            if localVarIndex is not None:
                putInstr2('stloc.'+str(localVarIndex))
            else:
                putInstr2('stsfld '+getClRTypeString(x.tp)+' MainClass::'+x.name)
        else:
            argIndex = None
            ai = 0
            for argument in current_top_proc[2]:
#                     if argument.name is x.name and argument.tp is x.tp:
                if argument.tp is x.tp:
                    name = argument.name
                    argIndex = ai
                    break
                ai+=1
            if argIndex is not None:
                putInstr2('starg.s '+str(argIndex))
    else:
        putInstr2('stsfld '+getClRTypeString(x.tp)+' MainClass::'+x.name)

In [43]:
def arraySetup(x):
    selectorClassName = 'MainClass'
    if hasattr(x, 'name'):
        if x.lev > 0:
            loadVariableCLR(x)
        else:
            if hasattr(x, 'selectorClass'):
                selectorClassName = getClRTypeString(x.selectorClass)[6:]
                putInstr2('ldfld '+getClRTypeString(x.tp)+' '+selectorClassName+'::'+x.name)
            else:
                putInstr2('ldsfld '+getClRTypeString(x.tp)+' '+selectorClassName+'::'+x.name)
    else:
        loadVariableCLR(x)
    putInstr2('ldfld '+getClRTypeString(x.tp.base)+'[] '+getClRTypeString(x.tp)[6:]+'::arr')